# 挂载Google Drive 提前放好Traning dataset & Testing dataset

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


# 从GitHub下载, 安装requirements

In [8]:
!nvidia-smi # Test GPU
%cd /content
!rm -rf "Graduation" # 移除之前的文件夹
# 我的 Github repository for Graduation project BraTS2020 -- sea_comet
!git clone -b master https://github.com/sea-comet/Graduation # 指定分支为master
!pip install -r ./Graduation/requirements.txt

Thu Apr 28 10:19:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Training

In [7]:
%cd /content/Graduation
!rm -rf "Graduation/runs"  # 删除之前的runs文件夹
# !python -m src.train --width 48 --arch Atten_Unet --epochs 150 --debug --optim adam # debug 用
!python -m src.train --width 48 --arch Atten_Unet --epochs 150 --optim adam # 使用 Atten_Unet
# !python -m src.train --width 48 --arch Unet --epochs 150 --optim adam # 使用 Unet

/content/Graduation
Training device used:  cuda:0
Working with 1 GPUs
{'arch': 'Atten_Unet',
 'batch_size': 1,
 'debug': False,
 'dropout': 0.0,
 'epochs': 150,
 'exp_name': '20220428_100712__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_warm3__dropout0.0',
 'fold': 0,
 'full': False,
 'lr': 0.0001,
 'norm_layer': 'group',
 'optim': 'adam',
 'resume': '',
 'seed': 16111990,
 'start_epoch': 0,
 'val': 3,
 'warm': 3,
 'weight_decay': 0.0,
 'width': 48,
 'workers': 2}
Creating Atten_Unet model...

invalid literal for int() with base 10: ''
using default group number
model features:  [48, 96, 192, 384]
total number of trainable parameters 19252068

train dataset path : /content/drive/My Drive/Dataset/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData

Val dataset number of batch: 40
Train dataset number of batch: 160
start warm-up now! 3 epochs
train Epoch: [0][  1/160]	BatchTime 11.876 (11.876)	DataTime  9.962 ( 9.962)	Loss 0.9420 (0.9420)	Acc 0.0580 (0.0580)
train 

# 打包并下载runs文件夹 （包含Traning & Validation结果 + Tensorboard图像+ segmentations nii文件）

In [ ]:
# 打包并下载
import os, tarfile
from google.colab import files
 
#逐个添加文件打包，未打包空子目录。可过滤文件。
#如果只打包不压缩，将"w:gz"参数改为"w:"或"w"即可。
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)

make_targz_one_by_one('runs.tar', '/content/Graduation/runs') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Testing (用额外的testing dataset -- 无seg.nii)

In [ ]:
%cd /content/Graduation
# 需要改改yaml地址，注意！！下面是个例子
!python -m src.test --config "/content/Graduation/runs/20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse/20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse.yaml" --devices 0 --mode val  


/content/Graduation
Working with 1 GPUs
config file:  /content/Graduation/runs/20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse/20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse.yaml
old args:  namespace(arch='Atten_Unet', batch_size=1, ckpt=PosixPath('/content/Graduation/runs/20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse/model_best.pth.tar'), com=None, debug=False, deep_sup=False, dropout=0.0, epochs=150, exp_name='20220409_051731__fold0_Atten_Unet_48_batch1_optimadam_adam_lr0.0001-wd0.0_epochs150_deepsupFalse_fp16_warm3__normgroup_dropout0.0_warm_restartFalse', fold=0, full=False, lr=0.0001, no_fp16=False, norm_layer='group', normalisation='minmax', optim='adam', resume='', seed=16111990, s

# Tensorboard

In [ ]:
# 先安装Tensorborad X
!pip install tensorboardX

In [ ]:
# 查看Tensorboard图像
%load_ext tensorboard
%tensorboard --logdir "/content/Graduation/runs"

# 打包并下载preds 文件夹 （包含testing dataset 40个预测结果 segmentations nii文件）

In [ ]:
# 打包并下载
import os, tarfile
from google.colab import files
 
#逐个添加文件打包，未打包空子目录。可过滤文件。
#如果只打包不压缩，将"w:gz"参数改为"w:"或"w"即可。
def make_targz_one_by_one(output_filename, source_dir):
  tar = tarfile.open(output_filename,"w")
  for root,dir_name,files_list in os.walk(source_dir):
    for file in files_list:
      pathfile = os.path.join(root, file)
      tar.add(pathfile)
  tar.close()
 
  files.download(output_filename)

make_targz_one_by_one('preds.tar', '/content/Graduation/preds') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>